# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
geoapify_key

'9939133a3a5449f29bc4e93d5794ddb8'

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
# Create a Pandas DF and use pd.read to load into a CSV
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
# Display the top 5 
city_data_df.head()

,City_ID,City,lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,boende,-0.2167,20.8667,296.01,93,100,0.68,CD,1695845230
1,1,tambacounda,13.7539,-13.7586,305.18,74,0,1.03,SN,1695845230
2,2,ushuaia,-54.8000,-68.3000,278.96,45,0,8.23,AR,1695845231
3,3,yarmouth,41.7057,-70.2286,291.23,73,0,6.26,US,1695845231
4,4,codrington,-38.2667,141.9667,280.56,96,24,3.62,AU,1695845108


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
# Set parameters: Geo, size,scale, color, alpha (opacity) , tiles, frame_width, frame_height

map_plot = city_data_df.hvplot.points(
    "Lng",
    "lat",
    geo=True,
    size="Humidity",
    scale=1,
    color="City",
    alpha=0.6,
    tiles="OSM",
    frame_width=700,
    frame_height=500
    
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Narrow down cities that fit criteria and drop any results with null values
# Create a new narrowred data frame for your ideal weather 
# Set a range of Max Temp, Wind Speed , and Cloudiness
# YOUR CODE HERE
narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 25) & (city_data_df['Max Temp'] > 19) & (city_data_df["Wind Speed"] < 2.5)   & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
# Use the Dropna() function to get rid of null values 
# YOUR CODE HERE
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

,City_ID,City,lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# Create a new DataFrame and indext the values you want from the previous
# YOUR CODE HERE
hotel_df = narrowed_city_df[["City","Country", "lat", "Lng" , "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# Add the Column wanted in square brackets then put a space assignment in place of the name
# YOUR CODE HERE
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,lat,Lng,Humidity,Hotel Name


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [19]:
# Set parameters to search for a hotel
# Find Hotel accomodations 
# Find limits 
# Use Geoapify Key
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey" : geoapify_key,
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # Assign the values from indexing
    # YOUR CODE HERE
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle: {longitude}, {latitude},{radius}"
    params["bias"] = f"proximity:{longitude}, {latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    # request.get(base_url,params=params) - to get the data within the parameters
    name_address = request.get(base_url, params=params)
    
    # Convert the API response to JSON format
    # Use .Json for formatting
    name_address = name_address.json
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


,City,Country,lat,Lng,Humidity,Hotel Name


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:
%%capture --no-display

# Configure the map plot
# Use the same map plot configurations 
# Add hover_cols for hotel and country.
map_plot = city_data_df.hvplot.points(
    "Lng",
    "lat",
    geo=True,
    size="Humidity",
    scale=1,
    color="City",
    alpha=0.5,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    hover_cols = ["Hotel", "Country"]
    
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,lat]   (City,Humidity,Country)